# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 15 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

In [ ]:
#!nvidia-smi

# Libraries

In [ ]:
!pip install torchsummaryX wandb --quiet

     |████████████████████████████████| 1.8 MB 2.1 MB/s 
     |████████████████████████████████| 162 kB 59.7 MB/s 
     |████████████████████████████████| 181 kB 37.1 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 158 kB 36.9 MB/s 
     |████████████████████████████████| 157 kB 61.1 MB/s 
     |████████████████████████████████| 157 kB 59.8 MB/s 
     |████████████████████████████████| 157 kB 53.9 MB/s 
     |████████████████████████████████| 157 kB 41.2 MB/s 
     |████████████████████████████████| 157 kB 39.1 MB/s 
     |████████████████████████████████| 157 kB 55.5 MB/s 
     |████████████████████████████████| 157 kB 36.9 MB/s 
     |████████████████████████████████| 156 kB 49.8 MB/s 


In [ ]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import sklearn.metrics
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
import glob
import librosa
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Thu Sep 29 20:17:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    30W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
### PHONEME LIST
PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"ripcurl11","key":"a924e45910075179ad325ad28d952008"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 1.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73275 sha256=0dd9fefcf2378b73598cf404d91abe96d04d46dfb1d25378531d0473aab8ddf2
  Stored in directory: /root/.cache/pip/wheels/de/f7/d8/c3902cacb7e62cb611b1ad343d7cc07f42f7eb76ae3a52f3d1
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
# commands to download data from kaggle

!kaggle competitions download -c 11-785-f22-hw1p2
!mkdir '/content/data'

!unzip -qo '11-785-f22-hw1p2.zip' -d '/content/data'

100% 2.12G/2.13G [00:32<00:00, 62.9MB/s]
100% 2.13G/2.13G [00:32<00:00, 69.4MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Augmentation

In [ ]:
class AugmentData():
    def __init__(self, sampling_rate=None, pitch_factor=None, speed_factor=None):
        self.sampling_rate = sampling_rate
        self.pitch_factor = pitch_factor
        self.speed_factor = speed_factor
                
    def inject_noise(self, data, noise_factor):
        self.data = data
        self.noise_factor = noise_factor 
        noise = np.random.randn(*self.data.shape)
        augmented_data = self.data + (self.noise_factor * noise)
        # Cast back to same datatype
        augmented_data = augmented_data.astype(type(self.data[0]))
        return augmented_data

    # def time_shift(self, data, sampling_rate, shift_max, shift_direction):
    #     shift = np.random.randint(sampling_rate * shift_max)
    #     if shift_direction == 'right':
    #         shift = -shift
    #     elif shift_direction == 'both':

    def change_pitch(self, data):
        self.data = data
        return librosa.effects.pitch_shift(self.data, self.sampling_rate, self.pitch_factor)

    def change_speed(self, data):
        self.data = data
        return librosa.effects.time_stretch(self.data, self.speed_factor)

    



In [ ]:
#aug_data_obj = AugmentData()

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [ ]:
!pwd

/content


In [ ]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, context=0, offset=0, partition= "train", limit=-1, percent_data=100): # Feel free to add more arguments

        self.context = context
        self.offset = context
        self.data_path = data_path

        paths = dict(
            train= '/train-clean-100/',
            dev= '/dev-clean/',
            test= '/test-clean/',
        )

        self.mfcc_dir = self.data_path + paths[partition] + 'mfcc' # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data
        self.transcript_dir = self.data_path + paths[partition] + 'transcript'  # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data
        #print('self.mfcc_dir: ', self.mfcc_dir)


        mfcc_names = sorted(os.listdir(self.mfcc_dir)) # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load
        
        transcript_names = sorted(os.listdir(self.transcript_dir)) # TODO: List files in self.transcript_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load

        assert len(mfcc_names) == len(transcript_names) # Making sure that we have the same no. of mfcc and transcripts

        self.mfccs, self.transcripts = [], []

        num_exmpls = int(len(mfcc_names) * percent_data / 100)


       # create augmentation obj here
        aug_data_obj = AugmentData()

        # TODO:
        # Iterate through mfccs and transcripts
        for i in tqdm(range(0, num_exmpls)):
        #   Load a single mfcc
            mfcc = np.load(self.mfcc_dir + '/' + mfcc_names[i], allow_pickle=True)
            #print(mfcc[i])
            # augment here
            aug_probability = (np.random.random() + 1) % 2 
            percent_aug = 0.3
            if aug_probability < percent_aug:
                # call inject noise here
                mfcc = aug_data_obj.inject_noise(mfcc, noise_factor=np.mean(mfcc)*0.5)

        #!!!!!   Optionally do Cepstral Normalization of mfcc
            #mfcc = (mfcc - mfcc.mean())/mfcc.std().  #about axis 0 only
        #   Load the corresponding transcript
            transcript = np.load(self.transcript_dir + '/' + transcript_names[i], allow_pickle=True)[1:-1] # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
            # without traversing through the transcript?)
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        print('mfccs: ', len(self.mfccs))

        # NOTE:
        # Each mfcc is of shape T1 x 15, T2 x 15, ...
        # Each transcript is of shape (T1+2) x 15, (T2+2) x 15 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that the final shape is T x 15 (Where T = T1 + T2 + ...) 
        self.mfccs = np.concatenate(self.mfccs, axis = 0)

        # TODO: Concatenate all transcripts in self.transcripts such that the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts = np.concatenate(self.transcripts, axis = 0)

        # Hint: Use numpy to concatenate
        #self.length = len(self.mfccs)
        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, ((self.context, self.context),(0,0)), 'constant', constant_values=0) # verify

        # These are the available phonemes in the transcript
        self.phonemes = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
        # But the neural network cannot predict strings as such. Instead we map these phonemes to integers
        
        # Map the phonemes to their corresponding list indexes in self.phonemes
        # # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)
        translations = {phoneme: idx for idx, phoneme in enumerate(self.phonemes)}
        self.transcripts = np.array([translations[phoneme] for phoneme in self.transcripts]) 

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs) - 2 *self.context
        print(f'mfccs shape: {self.mfccs.shape}')
        print(f'transcripts shape: {self.transcripts.shape}')

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        start_ind = ind + self.offset - self.context
        end_ind = ind + self.offset + self.context + 1
        frames = self.mfccs[start_ind : end_ind]

        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        
        frames = frames.flatten() # TODO: Flatten to get 1d data
        frames = torch.FloatTensor(frames) # Convert to tensors

        phoneme = torch.tensor(self.transcripts[ind], dtype=torch.long)       

        return frames, phoneme



In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, context=0, offset=0, partition= "test", limit=-1, percent_data=100): # Feel free to add more arguments

        self.context = context
        self.offset = context
        self.data_path = data_path

        paths = dict(
            train= '/train-clean-100/',
            dev= '/dev-clean/',
            test= '/test-clean/',
        )

        self.mfcc_dir = self.data_path + paths[partition] + 'mfcc/' # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data

        mfcc_names = sorted( filter( lambda x: os.path.isfile(os.path.join(self.mfcc_dir, x)),
                        os.listdir(self.mfcc_dir) ) ) # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load

        self.mfccs = []

        num_exmpls = int(len(mfcc_names) * percent_data / 100)
        # TODO:
        # Iterate through mfccs
        for i in tqdm(range(0, num_exmpls)):
        #   Load a single mfcc
            mfcc = np.load(self.mfcc_dir + '/' + mfcc_names[i], allow_pickle=True)
            #print(mfcc[i])

        #!!!!!   Optionally do Cepstral Normalization of mfcc
            #mfcc = (mfcc - mfcc.mean())/mfcc.std()

        #   Append each mfcc to self.mfcc
            self.mfccs.append(mfcc)

        # NOTE:
        # Each mfcc is of shape T1 x 15, T2 x 15, ...
        # Each transcript is of shape (T1+2) x 15, (T2+2) x 15 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that the final shape is T x 15 (Where T = T1 + T2 + ...) 
        self.mfccs = np.concatenate(self.mfccs, axis = 0)

        
        # Hint: Use numpy to concatenate
        #self.length = len(self.mfccs)
        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, ((self.context, self.context),(0,0)), 'constant', constant_values=0)


        # Length of the dataset is now the length of concatenated mfccs
        self.length = len(self.mfccs) - 2 *self.context

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        frames = self.mfccs[(ind + self.offset - self.context) : (ind + self.offset + self.context + 1)]

        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data
        frames = torch.FloatTensor(frames) # Convert to tensors
        
        return frames

    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments. 

In [ ]:
config = {
    'epochs': 30,
    'batch_size' : 2048,
    'context' : 35,
    'learning_rate' : 0.001,
    'architecture' : 'high-cutoff',
    'percent_data' : 100, ## added to control experiment size
    'hidden_sizes': [2400,2048,1900,1800,1500,1200] 
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [ ]:
data_path = '/content/data'

train_data = AudioDataset(data_path, context=config['context'], percent_data=config['percent_data']) #TODO: Create a dataset object using the AudioDataset class for the training data 

val_data = AudioDataset(data_path, partition='dev', context=config['context'], percent_data=config['percent_data']) # TODO: Create a dataset object using the AudioDataset class for the validation data 

test_data = AudioTestDataset(data_path, context=config['context'], percent_data=config['percent_data']) # TODO: Create a dataset object using the AudioTestDataset class for the test data 

100%|██████████| 28539/28539 [00:23<00:00, 1220.05it/s]


mfccs:  28539
mfccs shape: (36191204, 15)
transcripts shape: (36191134,)


100%|██████████| 2703/2703 [00:02<00:00, 1285.72it/s]


mfccs:  2703
mfccs shape: (1937566, 15)
transcripts shape: (1937496,)


100%|██████████| 2620/2620 [00:00<00:00, 2630.61it/s]


In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 4,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True)

val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                         batch_size=config['batch_size'], pin_memory= True,
                                         shuffle= False)

test_loader = torch.utils.data.DataLoader(test_data, num_workers= 2, 
                                          batch_size=config['batch_size'], pin_memory= True, 
                                          shuffle= False)


print("Batch size: ", config['batch_size'])
print("Context: ", config['context'])
print("Input size: ", (2*config['context']+1)*15)
print("Output symbols: ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  2048
Context:  35
Input size:  1065
Output symbols:  42
Train dataset samples = 36191134, batches = 17672
Validation dataset samples = 1937496, batches = 947
Test dataset samples = 1943253, batches = 949


In [ ]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    try:
        frames, phoneme = data
        print(frames.shape)
    except:
        print(f'error in {i}th batch')
    break
    #print(frames.shape, phoneme.shape)
    

torch.Size([2048, 1065])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [ ]:
from torch import nn

In [ ]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
class LinearBlock(nn.Module):
    '''
    stacks repetitive network layers together
    eg. linear -> relu
    '''
    def __init__(self, input_size, output_size, skip_conn=False):
        super().__init__()
        self.linear_block = nn.Sequential(
            nn.Linear(input_size, output_size, bias=False),
            nn.BatchNorm1d(output_size),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2)
        )
        self.skip_conn = skip_conn
    def forward(self, x):
        if self.skip_conn:
            return x + self.linear_block(x)
        else:
            return self.linear_block(x)

In [ ]:
import copy
def clone(module_name, hidden_sizes, N):
    return nn.ModuleList(
        [copy.deepcopy(module_name(hidden_sizes[i], hidden_sizes[i+1])) for i in range(N-1)]
        )

In [ ]:
class Network(nn.Module):
    def __init__(self, context):

        super().__init__()

        input_size = 15*(2*context + 1) #Why is this the case?
        hidden_sizes = config['hidden_sizes']
        output_size = 40 #Why?
        self.model = nn.Sequential(
            LinearBlock(input_size, hidden_sizes[0]),
            *clone(LinearBlock, hidden_sizes, N=len(hidden_sizes)),
            nn.Linear(hidden_sizes[-1], output_size)
        )
         

    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler. 

In [ ]:
input_size = 15*(2*config['context'] + 1)
model = Network(config['context']).to(device)

## load model here
#model.load_state_dict(torch.load('/content/drive/MyDrive/IDL_Adi/model_log_med_cutoff_with_8_layers.pth'))

frames, phoneme = next(iter(train_loader))
# Check number of parameters of your network - Remember, you are limited to 20 million parameters for HW1 (including ensembles)
summary(model, frames.to(device))

                                       Kernel Shape  Output Shape   Params  \
Layer                                                                        
0_model.0.linear_block.Linear_0        [1065, 2400]  [2048, 2400]   2.556M   
1_model.0.linear_block.BatchNorm1d_1         [2400]  [2048, 2400]     4.8k   
2_model.0.linear_block.ReLU_2                     -  [2048, 2400]        -   
3_model.0.linear_block.Dropout_3                  -  [2048, 2400]        -   
4_model.1.linear_block.Linear_0        [2400, 2048]  [2048, 2048]  4.9152M   
5_model.1.linear_block.BatchNorm1d_1         [2048]  [2048, 2048]   4.096k   
6_model.1.linear_block.ReLU_2                     -  [2048, 2048]        -   
7_model.1.linear_block.Dropout_3                  -  [2048, 2048]        -   
8_model.2.linear_block.Linear_0        [2048, 1900]  [2048, 1900]  3.8912M   
9_model.2.linear_block.BatchNorm1d_1         [1900]  [2048, 1900]     3.8k   
10_model.2.linear_block.ReLU_2                    -  [2048, 1900

/usr/local/lib/python3.7/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.0.linear_block.Linear_0,"[1065, 2400]","[2048, 2400]",2556000.0,2556000.0
1_model.0.linear_block.BatchNorm1d_1,[2400],"[2048, 2400]",4800.0,2400.0
2_model.0.linear_block.ReLU_2,-,"[2048, 2400]",NaN,NaN
3_model.0.linear_block.Dropout_3,-,"[2048, 2400]",NaN,NaN
4_model.1.linear_block.Linear_0,"[2400, 2048]","[2048, 2048]",4915200.0,4915200.0
5_model.1.linear_block.BatchNorm1d_1,[2048],"[2048, 2048]",4096.0,2048.0
6_model.1.linear_block.ReLU_2,-,"[2048, 2048]",NaN,NaN
7_model.1.linear_block.Dropout_3,-,"[2048, 2048]",NaN,NaN
8_model.2.linear_block.Linear_0,"[2048, 1900]","[2048, 1900]",3891200.0,3891200.0


In [ ]:
model

Network(
  (model): Sequential(
    (0): LinearBlock(
      (linear_block): Sequential(
        (0): Linear(in_features=1065, out_features=2400, bias=False)
        (1): BatchNorm1d(2400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Dropout(p=0.2, inplace=False)
      )
    )
    (1): LinearBlock(
      (linear_block): Sequential(
        (0): Linear(in_features=2400, out_features=2048, bias=False)
        (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Dropout(p=0.2, inplace=False)
      )
    )
    (2): LinearBlock(
      (linear_block): Sequential(
        (0): Linear(in_features=2048, out_features=1900, bias=False)
        (1): BatchNorm1d(1900, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Dropout(p=0.2, inplace=False)
      )
    )
    (3): LinearBlock(
      (linear_block

In [ ]:
criterion = torch.nn.CrossEntropyLoss() #Defining Loss function 
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate']) #Defining Optimizer
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=1,verbose=True)
# Recommended : Define Scheduler for Learning Rate, including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
# You can refer to Pytorch documentation for more information on how to use them.

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

3385

In [ ]:
from tqdm import tqdm

In [ ]:
def train(model, optimizer, criterion, dataloader):

    model.train()
    train_loss = 0.0 #Monitoring Loss
    
    for iter, (mfccs, phonemes) in enumerate(tqdm(dataloader)):

        ### Move Data to Device (Ideally GPU)
        mfccs = mfccs.to(device)
        phonemes = phonemes.to(device)

        ### Forward Propagation
        logits = model(mfccs)

        ### Loss Calculation
        loss = criterion(logits, phonemes)
        train_loss += loss.item()

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()
  
    train_loss /= len(dataloader)
    return train_loss

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode

    phone_true_list = []
    phone_pred_list = []
    # sil_count = 0
    # least_count = 0
    eval_loss_tot = 0.0

    for i, data in enumerate(dataloader):

        frames, phonemes = data
        ### Move data to device (ideally GPU)
        frames, phonemes = frames.to(device), phonemes.to(device) 

        with torch.inference_mode(): # makes sure that there are no gradients computed as we are not training the model now
            ### Forward Propagation
            logits = model(frames)

        ### Get Predictions
        predicted_phonemes = torch.argmax(logits, dim=1)
        
        ### Store Pred and True Labels
        phone_pred_list.extend(predicted_phonemes.cpu().tolist())
        phone_true_list.extend(phonemes.cpu().tolist())  



        eval_loss = criterion(logits, phonemes)
        eval_loss_tot += eval_loss.item()
        
        # Do you think we need loss.backward() and optimizer.step() here?
    
        del frames, phonemes, logits
        torch.cuda.empty_cache()
    
    scheduler.step(eval_loss_tot)

    ### Calculate Accuracy
    accuracy = sklearn.metrics.accuracy_score(phone_pred_list, phone_true_list) 
    # print('SIL instances: ',  sil_count)
    return accuracy*100

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb. 

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [ ]:
wandb.login(key="2178c9f0d96e90016c3d36bcccb07de5e0c51edc") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name = "first-run", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit=True, ### Allows reinitalizing runs when you re-run this cell
    project="hw1p2", ### Project should be created in your wandb account 
    config=config ### Wandb Config for your run
)

In [ ]:
### Save your model architecture as a string with str(model) 
model_arch = str(model)

### Save it in a txt file 
arch_file = open("model_arch.txt", "w")
file_write = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20220930_133028-3718tqzn/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()

best_acc = 0.0 ### Monitor best accuracy in your run

save_path = '/content/drive/MyDrive/IDL_Adi/model_log_high_cutoff_with_data_aug.pth'              ########### Correct for overwriting

for epoch in range(config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    train_loss = train(model, optimizer, criterion, train_loader)
    accuracy = eval(model, val_loader)

    print("\tTrain Loss: {:.4f}".format(train_loss))
    print("\tValidation Accuracy: {:.2f}%".format(accuracy))


    ### Log metrics at each epoch in your run - Optionally, you can log at each batch inside train/eval functions (explore wandb documentation/wandb recitation)
    wandb.log({"train loss": train_loss, "validation accuracy": accuracy})

    ### Save checkpoint if accuracy is better than your current best
    if accuracy >= best_acc:

      ### Save checkpoint with information you want
      torch.save({'epoch': epoch,
              'model_state_dict': model.state_dict(),
            #   'optimizer_state_dict': optimizer.state_dict(),
              'loss': train_loss,
              'acc': accuracy}, 
        './model_checkpoint.pth')
      
      ### Save checkpoint in wandb
      wandb.save('checkpoint.pth')

      torch.save({'epoch': epoch,
              'model_state_dict': model.state_dict(),
            #   'optimizer_state_dict': optimizer.state_dict(),
              'loss': train_loss,
              'acc': accuracy}, save_path)

    # Is your training time very high? Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
    # Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

### Finish your wandb run

run.finish()


Epoch 1/30


100%|██████████| 17672/17672 [12:50<00:00, 22.94it/s]


	Train Loss: 0.7085
	Validation Accuracy: 80.43%

Epoch 2/30


100%|██████████| 17672/17672 [12:52<00:00, 22.89it/s]


	Train Loss: 0.5387
	Validation Accuracy: 82.57%

Epoch 3/30


...

100%|██████████| 17672/17672 [12:42<00:00, 23.17it/s]


	Train Loss: 0.4403
	Validation Accuracy: 84.22%

Epoch 6/30


100%|██████████| 17672/17672 [12:36<00:00, 23.37it/s]


	Train Loss: 0.4252
	Validation Accuracy: 84.46%

Epoch 7/30


 78%|███████▊  | 13716/17672 [09:42<02:42, 24.40it/s]

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *random_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
  ### What you call for model to perform inference?
  model.eval()

  ### List to store predicted phonemes of test data
  test_predictions = []

  ### Which mode do you need to avoid gradients?
  with torch.no_grad():

      for i, frames in enumerate(tqdm(test_loader)):

          frames = frames.float().to(device)             
          
          output = model(frames)

          ### Get most likely predicted phoneme with argmax
          predicted_phonemes = torch.argmax(output, dim=1)        ###      !!!!!!!!!     Cross-Check

          ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
          test_predictions.extend(predicted_phonemes.cpu().tolist())  #using .cpu() to copy tensors to host cpu first ###      !!!!!!!!!     Cross-Check
          
  return test_predictions

In [ ]:
predictions = test(model, test_loader)

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
## Submit to kaggle competition using kaggle API
#!kaggle competitions submit -c 11-785-f22-hw1p2 -f ./submission.csv -m "Test Submission"